In [11]:
import pandas as pd
import numpy as np
import pathlib
import os
import sys
import glob
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from IPython.display import Image, display
import cv2
%matplotlib inline

## Preprocess data

In [29]:
data_path = "./data"
data_p = pathlib.Path(data_path)

In [30]:
meta_table = pd.read_csv("./_filename_dict.csv")

In [31]:
# Filter is_readable_image == False
f_table = meta_table[meta_table['is_readable_image']].copy()
# f_table = f_table[f_table['_format'] != ".gif"].copy()
f_table.reset_index(drop=True, inplace=True)
f_table.fillna("", inplace=True)

In [32]:
filenames = ['./data/' + fname for fname in f_table['_filename'].tolist()]

def preprocess_image(image):
    image = tf.image.decode_jpeg(image)
    return image

def load_and_preprocess_image(path):
    image = tf.io.read_file(path)
    return preprocess_image(image)

def get_corrupted_image_files(file_list):
    cr_list = []
    
    for i, f in enumerate(filenames):
        try:
            load_and_preprocess_image(f)
        except Exception as e:
            cr_list.append((i, f))
            
    return cr_list

corrupted_images = get_corrupted_image_files(filenames)

In [33]:
corrupted_images_index = [i[0] for i in corrupted_images]
corrupted_images_index

[1384, 4255, 5142, 6098, 7621, 7910]

In [34]:
f_table = f_table.iloc[~f_table.index.isin(corrupted_images_index)].copy()
f_table.reset_index(drop=True, inplace=True)
f_table.fillna("", inplace=True)

In [35]:
f_table['level0'].value_counts()

湿垃圾     3703
可回收物    3629
有害垃圾    2806
干垃圾     2407
Name: level0, dtype: int64

In [36]:
f_table.head()

,_filename,_format,sample,spider_id,level0,level1,is_readable_image
0,干垃圾_编织袋_36.jpeg,.jpeg,编织袋,36,干垃圾,,True
1,湿垃圾_食材废料_水产_91.jpg,.jpg,水产,91,湿垃圾,食材废料,True
2,湿垃圾_过期食品_肉干_7.jpg,.jpg,肉干,7,湿垃圾,过期食品,True
3,干垃圾_橡皮泥_20.jpg,.jpg,橡皮泥,20,干垃圾,,True
4,有害垃圾_废含汞温度计、废含汞血压计_水银体温计_84.jpg,.jpg,水银体温计,84,有害垃圾,废含汞温度计、废含汞血压计,True


In [37]:
filenames = ['./data/' + fname for fname in f_table['_filename'].tolist()]
lv0_labels = f_table['level0'].tolist()

In [38]:
labelencoder = preprocessing.LabelEncoder()
lv0_labels_encoded = labelencoder.fit_transform(lv0_labels)

In [39]:
print(labelencoder.transform(['可回收物', "有害垃圾", "湿垃圾", "干垃圾"]))
print(len(labelencoder.classes_))

lv0_n_classes = len(labelencoder.classes_)

[0 2 3 1]
4


In [40]:
raw_filenames, test_filenames, raw_labels, test_labels = train_test_split(filenames
                                                                        , lv0_labels_encoded
                                                                        , train_size=0.9
                                                                        , random_state=42)

train_filenames, val_filenames, train_labels, val_labels = train_test_split(raw_filenames
                                                                        , raw_labels
                                                                        , train_size=0.9
                                                                        , random_state=42)

In [41]:
print(len(train_filenames), len(train_labels))
print(len(val_filenames), len(val_labels))
print(len(test_filenames), len(test_labels))

10161 10161
1129 1129
1255 1255


In [42]:
train_data = tf.data.Dataset.from_tensor_slices(
  (tf.constant(train_filenames), tf.constant(train_labels))
)
val_data = tf.data.Dataset.from_tensor_slices(
  (tf.constant(val_filenames), tf.constant(val_labels))
)

In [43]:
IMAGE_SIZE = 96
BATCH_SIZE = 32

In [44]:
def _parse_fn(filename, label):
    try:
        # read file as byte
        image_string = tf.io.read_file(filename)
        # decode as string 
        image_decoded = tf.image.decode_jpeg(image_string)
        # image_decoded = tf.image.decode_gif(image_string)
        ## P.S tf.image.decode_image return shapeless tensor.
        # image_decoded = tf.image.decode_image(image_string)
        
        # normailization
        image_normalized = (tf.cast(image_decoded, tf.float32)/127.5) - 1
        # resize to given image size
        image_resized = tf.image.resize(image_normalized, (IMAGE_SIZE, IMAGE_SIZE))
        return image_resized, label
    except Exception as e:
        print(e)
        print(filename)

In [45]:
train_data = (train_data.map(_parse_fn)\
            .shuffle(buffer_size=10000)\
            .batch(BATCH_SIZE)
             )

val_data = (val_data.map(_parse_fn)
           .shuffle(buffer_size=10000)
           .batch(BATCH_SIZE)
           )

## Define the model

In [46]:
# Base model with MobileNetV2
IMG_SHAPE = (IMAGE_SIZE, IMAGE_SIZE, 3)

base_model = tf.keras.applications.InceptionV3(input_shape=IMG_SHAPE,
                                               include_top=False, 
                                               weights='imagenet')
base_model.trainable = False

In [47]:
# Trainable classification head
maxpool_layer = tf.keras.layers.GlobalMaxPooling2D()
avgpool_layer = tf.keras.layers.GlobalAveragePooling2D()
dropout_layer = tf.keras.layers.Dropout(0.5)
prediction_layer = tf.keras.layers.Dense(lv0_n_classes, activation='softmax')

learning_rate = 1e-4

model = tf.keras.Sequential([
    base_model
    , avgpool_layer
    # , dropout_layer
    , prediction_layer
])

# Now the optimizer is under tf.keras.
model.compile(optimizer=tf.keras.optimizers.Adam(lr=learning_rate),
             loss='categorical_crossentropy',
             metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_v3 (Model)         (None, 1, 1, 2048)        21802784  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 4)                 8196      
Total params: 21,810,980
Trainable params: 8,196
Non-trainable params: 21,802,784
_________________________________________________________________


## Train model

In [48]:
num_epochs = 1
steps_per_epoch = round(len(train_filenames))//BATCH_SIZE
val_steps = 20

In [49]:
history = model.fit(train_data.repeat(),
                    epochs=num_epochs,
                    steps_per_epoch = steps_per_epoch,
                    validation_data=val_data.repeat(), 
                    validation_steps=val_steps)

  1/317 [..............................] - ETA: 5:02:18 - loss: 8.7964 - accuracy: 0.1562

InvalidArgumentError: Cannot batch tensors with different shapes in component 0. First element had shape [96,96,3] and element 1 had shape [96,96,4]. [Op:IteratorGetNextSync]